# OpenMSI Arrayed Analysis Tool
This Notebook is meant as an example for those who prefer programmatic access to the OpenMSI Arrayed Analysis toolkit over the interactive dialogs in the other notebook

***PLEASE NOTE THIS EXAMPLE IS MEANT FOR EXPERIENCED PYTHON PROGRAMMERS.***

***If you want a no-fuss data analysis tool, use the notebook called <a href="OpenMSI_Arrayed_Analysis_with_dialogs.ipynb">OpenMSI_Arrayed_Analysis_with_dialogs.ipynb</a>.***

There is, of course, more than one way to leverage this tool, so use this example to inspire your analysis pipeline. You might want to mix-and-match interactive commands from the interactive notebook with some non-interactive code featured here.

This way of accessing the toolbox is still not entirely non-interactive, because logging in and spot grid definition (the roughPosition function) are ***always*** interactive

Avoid in-lining matplotlib in this notebook, because the spot placement GUI requires windowed mode.
If you do need to use inline graphics, save the img using pickle, and un-pickle it in a fresh notebook in which you've also %run Arrayed_Analysis_Tools.ipy

In [22]:
#load the code. Since it's specialized ipython notebook code, use '%run' rather than 'import'
%run Arrayed_Analysis_Tools.ipy

Completed loading OpenMSI Arrayed Analysis Toolkit


In [2]:
#OPTIONAL: log into OpenMSI.nersc.gov
openMSIsession = login()

Login appears to be successful!


In [3]:
#If you choose NOT to log in, you can create a new openMSISession
if "openMSIsession" not in locals():
    openMSIsession=OpenMSIsession()

In [4]:
#Use a file selector to pick a file to download
openMSIsession.fileSelector() #if this is not the last statement of the cell,
                              #you'll need to call IPython.display.display() on this
#fileSelector() stores the picked file in the openMSI object
#so that it downloads the right file in the getArrayedImage function later

Stored 'arrayed_analysis_default_filename' (unicode)
Stored 'arrayed_analysis_default_filename' (unicode)
Stored 'arrayed_analysis_default_filename' (unicode)


In [9]:
#generates an ArrayedImage from these specificied ions.
myIons = [351.25, 212.05, 137.04]
img=openMSIsession.getArrayedImage(myIons,0.3,massRangeReductionStrategy=PeakArea())
# Look at this function's docstring to find explanations of all the arguments, or use the defaults
# MassReductionStrategies available are PeakArea() PeakHeight() and AreaNearPeak(), PeakArea() is the default
# If you don't like to use the fileselector in the previous cell, you can pass a filename to the getArrayedImage function

loading ion 1 of 3. m/z = 351.250000 +/- 0.300000
Time to load ion: 0.588999986649 seconds
loading ion 2 of 3. m/z = 212.050000 +/- 0.300000
Time to load ion: 0.735000133514 seconds
loading ion 3 of 3. m/z = 137.040000 +/- 0.300000
Time to load ion: 1.06699991226 seconds
Image has been loaded.


In [10]:
#optional: check what the image looks likes now

plt.imshow(img.baseImage,cmap='jet_r',clim=(0.0,np.amax(img.baseImage)/2)) #get rid of the /2 to see a wider range, or divide
                                                                           #by a bigger number if you want a narrower range
plt.colorbar() 
putWindowOnTop()
plt.show()

In [15]:
#define spot centers as a trapezoid. Set HexagonalOffset = 0.5 or -0.5 for dense circle packing or False for square packing
columns = 20
rows = 12
img.roughPosition(columns,rows,colormap='jet_r',hexagonalOffset=-0.5)

4
new spot x and y locations have been saved.


In [16]:
#automagically optimize the spot centers to correspond to the actual spots on the image
img.optimizeSpots(verbose=True,minimumScore=300,halfboxsize=2,integrationRadius=2,avoidOverlaps=True,overlapDistance_squared=3)

#if you don't want spots to move off the grid unless they actually find a spot, set a minimumScore.
#(this is useful is there's missing spots from your grid)
#if you want to find out what a reasonable number for a minimumScore is, try running the optimization with verbose=True,
#or use the img.optimizeSpots_with_dialogs() function.
#conversely, if you want to make /sure/ that there is a spot with a score of /at least/ minimumScore,
#you can choose to set raiseExceptions=True

13% done with the optimization process
27% done with the optimization process
done with optimization round 1 of 3
total score: 6012263.43423 	 average spot score: 25051.0976426
low spot score: 316.307692308 	 high spot score: 175110.307692
41% done with the optimization process
55% done with the optimization process
done with optimization round 2 of 3
total score: 10484697.6392 	 average spot score: 43686.2401634
low spot score: 324.0 	 high spot score: 257392.538462
69% done with the optimization process
83% done with the optimization process
97% done with the optimization process
done with optimization round 3 of 3
total score: 12581081.9424 	 average spot score: 52421.17476
low spot score: 327.538461538 	 high spot score: 310156.307692
optimization routine completed. new spot x and y positions saved.


In [20]:
#check the positions of the spots and manually adjust them if need be
img.fineTunePosition(colormap='jet_r',markerRadius=3,spotLabelsAlwaysOn=True)

new spot x and y locations have been saved.


In [21]:
#Optional: Save the ArrayedImage into a pickle file.
import pickle
pickle.dump(img, open("saved_image","wb"))

In [23]:
#Optional: Load an ArrayedImage from a pickle file. This way you can work off-line
import pickle
img=pickle.load(open("saved_image","rb"))
print img

ArrayedImage based on /project/projectdirs/openmsi/omsi_data_private/raad0102/20160216MdR_Maldi_Secondary_metabolite_library.h5
Ions loaded: [351.25, 212.05, 137.04]
# of spot locations defined: 240
# of spot pixel masks defined: 240


In [24]:
#all the rest of the functions can be run non-interactively. In this example it will save the results to a tab-separated file
#as well as to a pandas data-frame for data analysis directly in this notebook

#You'll need to call this function. It returns a list of spots (where each spot is a list of pixels),
#which is also stored inside the object.
spots=img.generateSpotList(integrationRadius=2)

#Write results to a file
#if you don't pass it an explicit spotList it will use the spot set stored in the ArrayedImage
img.writeResultTable("resultTable.csv",minPixelIntensity=1000,alphaRows=True)

df=img.resultsDataFrame(minPixelIntensity=0,alphaRows=True) #generate the dataframe
#alphaRows=True makes the indexes of the data frame strings with an alphabetical row identifier.
#alphaRows=False sets the indexes to 2-tuples (row,column)

IPython.display.display(df)
sums_df=df.loc[:,(slice(None),'sum')] #get the sums from the dataframe
sums_df.columns = sums_df.columns.get_level_values(0) #name the columns to make indexing easier later
percentage_firstion=100.0*sums_df[img.ions[0]]/sums_df.sum(axis=1) #calculate the percentage
percentage_firstion.sort() #rank the data from low to high. Using sort() instead of sort_values(inplace=True) for backward compatibility
plt.bar(range(len(percentage_firstion)),percentage_firstion,edgecolor='b') #define a bar chart
plt.xlabel('Spot rank') #set x axis label
plt.ylabel("% m/z={:.1f} of all loaded ions".format(img.ions[0])) #set y axis label
plt.xlim(0,len(percentage_firstion)-1) #set x axis range
plt.show()



240 spots generated. number of spots with N pixels:{10: 2, 11: 43, 12: 48, 13: 96, 14: 51}


C:\Users\trist_000\PycharmProjects\OpenMSI_Arrayed_Analysis_Tools\resultTable.csv.csv

ion        351.25                                              212.05  \
descriptor    sum         mean  median  min   max num_pixels      sum   
A01          1937   149.000000   139.0   97   216         13    16812   
A02          6363   489.461538   493.0  194   778         13  1980152   
A03          6098   469.076923   478.0  266   732         13  2037523   
A04          1103    84.846154    83.0   68   118         13     1438   
A05          1065    88.750000    91.0   65   108         12     1290   
A06          1290    92.142857    93.0   69   116         14     1475   
A07          1091    90.916667    89.0   75   105         12     1252   
A08          1278    91.285714    99.0   58   111         14     1694   
A09          1274    91.000000    98.0   60   113         14     1571   
A10          1220   101.666667   103.5   80   124         12     1483   
A11          1088    83.692308    85.0   51   116         13     1410   
A12          1314   101.076923    94.0   79   134         13     1566   
A13          1281    98.538462    96.0   78   122         13     1664   
A14          1332   102.461538   102.0   78   138         13     1584   
A15           926    84.181818    83.0   51   106         11     1318   
A16          1116    85.846154    85.0   63   105         13     1522   
A17          1210    86.428571    82.0   67   111         14     1481   
A18          1012    92.000000    96.0   55   111         11     1150   
A19          1090    99.090909    96.0   85   120         11     1312   
A20          1113    85.615385    80.0   70   111         13     1537   
B01          2031   169.250000   163.5  105   320         12    25737   
B02          5168   469.818182   413.0  276   825         11  1570451   
B03          6238   479.846154   483.0  112   666         13  2108767   
B04          1214    86.714286    90.0   63   117         14     1642   
B05          1422   109.384615   107.0   88   132         13     1555   
B06          1237    95.153846   102.0   57   109         13     1575   
B07          1204   100.333333    96.5   72   134         12     1548   
B08          1364   104.923077    99.0   67   159         13     1441   
B09          1174    90.307692    89.0   71   121         13     1554   
B10          1166    89.692308    92.0   76   109         13     1440   
..            ...          ...     ...  ...   ...        ...      ...   
K11          5267   405.153846   334.0  122  1127         13  1070206   
K12          4163   320.230769   278.0  105   683         13   592190   
K13          4773   397.750000   373.0  218   674         12  1127845   
K14          6215   478.076923   450.0   90  1061         13  1650029   
K15          7118   508.428571   502.5  114  1064         14  1813145   
K16          6419   493.769231   486.0  101  1056         13  1299102   
K17          3253   325.300000   294.0   82   884         10   850860   
K18          6348   488.307692   514.0  102   852         13  1710991   
K19          7478   534.142857   414.5   99  1155         14  1764635   
K20          7607   585.153846   535.0   79  1509         13  1445742   
L01          2219   158.500000   140.5   90   296         14    57588   
L02          5360   412.307692   185.0   82  1586         13   670315   
L03          2436   203.000000   142.5   90   550         12   229930   
L04          5436   418.153846   474.0   57   897         13  1178321   
L05          6864   528.000000   484.0   99  1180         13  1379697   
L06          1294   117.636364    93.0   62   402         11    27054   
L07          4461   318.642857   208.5   91   724         14   674447   
L08          4014   308.769231   114.0   71  1077         13   397631   
L09          4818   401.500000   394.5   90   885         12   388283   
L10          5323   483.909091   530.0  116  1033         11   694488   
L11          3100   221.428571   122.5   60   922         14   187384   
L12         20172  1681.000000  1008.0   79  5069         

C:\bin\Anaconda\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


In [25]:
#Optional: If you want to see a visual respresenation of which pixels are taken into account when
#calculating the result table. If you don't pass it an explicit spotList it will use the stored spotList
maskedimg=img.showMaskedImage(alphaRows=True)
#if you don't want to see the spot row and column co-ordinates, use img.generateMaskedImage() in stead

240 spots generated. number of spots with N pixels:{10: 2, 11: 43, 12: 48, 13: 96, 14: 51}
